In [4]:
using MAT, Statistics, LinearAlgebra,Random, Plots

In [5]:
include("mvnrnd.jl")

mvnrnd (generic function with 1 method)

Reads in all the variables optained that are used in line 93 and onwards and are defined before that.

In [6]:
function string_as_varname(s::AbstractString,v::Any)
         s=Symbol(s)
         @eval (($s) = ($v))
end

file = matopen("simulationData.mat")



varnames = names(file) 
#[IO, price, Omega, data, labor, mu, aggtfp, alpha, grossy, stfp, beta, agggdp, Sigma, capital, vadd]
for varname in varnames
    val = read(file, varname)
    # Instantiate the variable with its name in the current scope
    string_as_varname(varname,val)
    end

close(file)

We now want to figure out what every variable stands for, and where it comes from.
So far we know:
- **IO**: Input-Output Matrix
- **STFP** Sectoral TFP Groth per anno
- **$\Omega$**: I think this is the same as in *Definition 3* Es gilt $$(\Omega)_i^j = \frac {(\mathit{IO})_i^j} {\sum_{k=0}^N(\mathit{IO}_k^j)}$$ Also der Anteil eines Sektors $i$ an der Gesamptproduktion des Sektors $j$
- **$\Sigma$**: Covarianz von SFTP
- **$\mu$**: Rowwise mean of stfp. So the mean annual groth of each sector
- **data**: quantity data matrix
- **price**: price data matrix. I think price and data are for our anyalsis quite useless, they are used as building blocks
- **grossy** nominal gross output
- **vadd** nominal value added
- **labor** nominal labor
- **capital** nominal captial
- **$\alpha$**: vadd / grossy. 

This is not used anywhere in the code.

In [7]:
cum_stfp = cumsum(log.(1 .+ stfp)',dims=1);
cum_stfp_4year = cum_stfp[:,begin:4:end];
covariance_4year = Statistics.cov(cum_stfp_4year,dims=2,corrected=false);

Here comes the randomness. As far as i can tell the generated vector $x$ has mean $\mu = \text{Var}(\mathit{stfp})$ and $\sigma = \text{Var}(\mathit{stfp})$ Because $\sigma$ is a Diagonal Matrix, there is no correlation between each random number, what makes this rather perculiar. Intuition wise I would say, that this implies, that the shocks are independent of one-another.

$$(A)_i^j = e^{x_i}$$

In [8]:
A = exp.(mvnrnd(-1/2*diag(Sigma),diagm(diag(Sigma))));

Let's write the formulas for everything down, in an hope to understand it better

$$q = (\Omega \cdot p^{1-\theta})^{\frac 1 {1-\theta}}$$
$$w_i = (p_i \cdot A_i^{\frac {1 - \epsilon}{\epsilon}} \cdot \alpha_i^{\epsilon^{-1}} * y_i^{\epsilon^{-1}} \cdot L^{-1 \epsilon^{-1}}) \quad 1 \leq i \leq N$$ 

In [ ]:
function [outineq, Out, outineq2, OutDeriv]=Simulation_Derivs(X, A, beta, Omega, alpha, epsilon, theta, sigma,L) % no reallocation of labor
    N = length(alpha);
    p = X(1:N);
    y = X(N+1:2*N);

    q = (Omega*(p.^(1-theta))).^(1/(1-theta));0
    w = p.*(A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(y.^(1/epsilon)).*(1./L).^(1/epsilon);
    C = w'*L;

    Out(1:N) = p - (diag(A)^(epsilon-1)*(alpha.*w.^(1-epsilon)+(1-alpha).*q.^(1-epsilon))).^(1/(1-epsilon));
    Out(N+1:2*N) = y' -y'*diag(p)^epsilon*diag(A)^(epsilon-1)*diag(q)^(theta-epsilon)*diag(1-alpha)*Omega*diag(p)^(-theta) - beta'*diag(p)^(-sigma)*C;
    Out = Out';
    outineq = [];
    outineq2 = [];

    DQDP = bsxfun(@times, (q.^theta), (p.^(-theta))').*Omega; % 
    DWDP = diag((A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(y.^(1/epsilon)).*(1./L).^(1/epsilon)); %checked
    DWDY = (1/epsilon)*diag(p.*(A.^((epsilon-1)/epsilon)).*(alpha.^(1/epsilon)).*(y.^(1/epsilon-1)).*(L).^(-1/epsilon)); %checked
    DCDP = DWDP'*L; %checked
    DCDY = DWDY'*L;%checked
    DOut1DP = eye(N) - diag(diag(A)^(-1)*((alpha.*(w.^(1-epsilon))+(1-alpha).*(q.^(1-epsilon)))).^(epsilon/(1-epsilon)))*...
        (diag(alpha)*diag(w.^(-epsilon))*DWDP+diag(1-alpha)*diag(q.^(-epsilon))*DQDP);

    DOut1DY =  -diag(diag(A.^(-1))*((alpha.*(w.^(1-epsilon))+(1-alpha).*(q.^(1-epsilon)))).^(epsilon/(1-epsilon)))*...
        (diag(alpha)*diag(w.^(-epsilon))*DWDY);

    DOut2DP = -(epsilon * diag(p.^(-theta))*Omega'*diag((p.^(epsilon-1)).*(y).*(q.^(theta-epsilon)).*(1-alpha).*(A.^(epsilon-1)))...
        +(theta-epsilon)*diag(p.^(-theta))*Omega'*diag((p.^(epsilon)).*(y).*(q.^(theta-epsilon-1)).*(1-alpha).*(A.^(epsilon-1)))*DQDP ...
        -sigma*diag(beta.*p.^(-sigma-1))*C+ bsxfun(@times, beta.*(p.^(-sigma)), DCDP')...
        - theta* diag(p.^(-theta-1)).*diag(Omega'*diag((p.^(epsilon)).*(q.^(theta-epsilon)).*(1-alpha).*(A.^(epsilon-1)))*y));

    DOut2DY = eye(N) - (diag(p)^epsilon*diag(A)^(epsilon-1)*diag(q)^(theta-epsilon)*diag(1-alpha)*Omega*diag(p)^(-theta))' - bsxfun(@times, beta.*(p.^(-sigma)), DCDY');
    OutDeriv = [DOut1DP DOut1DY; DOut2DP DOut2DY]';

end